In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_row", None)
import numpy as np
import re
import nltk
import glob
import string

In [2]:
# Function that calculates "points" for a card to be a "main entry" vs. "narrative," returns True if it's a "main entry"
def is_main_entry(index, row,df):
    # Check if previous and subsequent card authors match -> fix mispelled name in between
    if index > 0 and index < len(df)-1 and df.iloc[index-1]['Name'] == df.iloc[index+1]['Name'] and row['Name'] != df.iloc[index-1]['Name']:
        row['Name'] = df.iloc[index-1]['Name']
    # Check for new author -> automatic "main entry"
    elif index == 0 or row['Name'] != df.iloc[index-1]['Name']:
        return True
        
    # Calculate "points" for classification
    main_entry_pts, narrative_pts = 0, 0
    
    # Check length, longer -> narrative, shorter -> main entry
    if len(row['Text']) > 300:
        narrative_pts += 1
    elif len(row['Text']) < 200:
        main_entry_pts += 1
        
    # Check for misc. main entry identifiers ("GUIDE", "Section X", "X items") -> main entry
    if "Section" in row['Text']:
        main_entry_pts += 1
    if "items" in row['Text']:
        main_entry_pts += 1
    if "GUIDE" in row['Text'].upper() or "GUID" in row['Text'].upper() or "GUDE" in row['Text'].upper():
        main_entry_pts += 3
        
    # Check for misc. narrative identifiers
    if "FOR INFORMATION ABOUT THIS COLLECTION" in row['Text']:
        narrative_pts += 5
    if "Addition" in row['Text'] or "added" in row['Text']:
        narrative_pts += 1
    if not re.search(r"\d+", row['Text']):
        narrative_pts += 1
    if type(row['Text']) == str and type(df.iloc[index-1]['Text']) == str:
        # Matches begining of previous card
        if row['Text'][:50] == df.iloc[index-1]['Text'][:50]:
            narrative_pts += 1
        # Previous card indicates continuation of narrative onto current card
        if "(Continued on next card)" in df.iloc[index-1]['Text'] or "SEE NEXT CRD" in df.iloc[index-1]['Text'] or "Go to following card" in df.iloc[index-1]['Text']:
            narrative_pts += 5
        
    # Check for capitalized last name & card number at beginning of text (Card 2, JONES, Smith, John 3, etc.) -> narrative
    if re.search(r"Card [0-9]+", row['Text']):
        narrative_pts += 2
    elif (row['Name'].split(",")[0]).upper() in row['Text']:
        narrative_pts += 1
    elif re.search(r"{row['Name']} \d+", row['Text']):
        narrative_pts += 2
    
    # Tokenize card
    word_tokens = nltk.word_tokenize(row['Text'])
    tagged_text = nltk.pos_tag(word_tokens)
    
    # Check for "sentence format" (pos only present in narrative [verb, adj.]) -> narrative
    narr_pos = ["VB", "JJ", "PRP", "RB", "IN", "DT"]
    for word in tagged_text:
        if word[1] in narr_pos:
            narrative_pts += .3
    
    # Check for accessioned date (e.g., 13 MAR 97, 10-10-69) -> main entry
    if re.search(r"\d{1,2}-\d{1,2}-\d{2}", row['Text']):
        main_entry_pts += 1
    elif re.search(r"\d{1,2}\s[A-Z]{3}\s\d{2}", row['Text']):
        main_entry_pts += 2
    elif re.search(r"[A-Z]{3}\s\d{1,2}\s\d{4}", row['Text']):
        main_entry_pts += 2
        
#     print("{}: {} {}".format(index, main_entry_pts, narrative_pts))
        
    # Determine card classification
    if main_entry_pts >= narrative_pts:
        return True
    return False

In [3]:
def sort_collection(df):
    # Check every card and classify as "main entry" or "narrative", number by collection
    collection_num, collection_list = -1, []
    for index, row in df.iterrows():
        # Check for NaN values
        if pd.isna(row['Text']) or pd.isna(row['Name']) or row['Text'] == "" or row['Name'] == "":
            collection_list.append(-1)
        else:
            # Main entries signify a new collection
            if is_main_entry(index, row,df):
                collection_num += 1
            collection_list.append(collection_num)
    # Add column to dataframe with collections numbered
    df['Collection'] = collection_list
    # Label by collection headers as index, with original index
    df["Page_No_Within_Drawer"] = df.index
    curr = -100
    collection_header = []
    for index,row in df.iterrows():
        if(row["Collection"]!= curr):
            collection_header.append(row["Name"])
        else:
            collection_header.append("")
        curr = row["Collection"]
    df["Collection_Head"] = collection_header
    return df

In [4]:
# Sort all drawers and combine them
file_list_name = (glob.glob(r"catalog\file_name_checked/*"))
file_list = [pd.read_csv(i) for i in file_list_name]
sorted_list = [sort_collection(i) for i in file_list]
all_text_pre = pd.concat(sorted_list)

In [5]:
all_text_pre.to_csv("all_sorted_before_year_column.csv",index=False)

In [25]:
all_text = pd.read_csv(r"catalog\all_sorted_before_year_column.csv")

In [26]:
# add an is_collection_header simple column to label easily
li = []
n = -100
for i in all_text.Collection.values:
    if(n==-1):
        li.append(0)
    elif(n!=i):
        li.append(1)
    else:
        li.append(0)
    n=i
all_text["Is_collection_head"] = li
all_text.head()


,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,NaN,0


In [27]:
# Create a year column for main entries
main_entries =  all_text[all_text["Is_collection_head"]==1]
ind_main_entries = all_text[all_text["Is_collection_head"]==1].index


# patterns used to extract sequentially if one does not exist
first = "(1\d{3}[-]\s?\d{2,4})"
second = "(1\d{3}.{0,9}1\d{0,3})"
third = "(1\d{3})"
fourth = "(1\d{3}[-]c(a)?\\.)"
fifth = "(n\\.\sd\\.)"    #no date

year_vals = main_entries.Text.str.extract(first)[0].fillna(( # if it returns na, find next pattern
    main_entries.Text.str.extract(second)[0])).fillna((
        main_entries.Text.str.extract(third)[0])).fillna((
        main_entries.Text.str.extract(fourth)[0])).fillna((
        main_entries.Text.str.extract(fifth)[0])).values

all_text["Year"] =np.nan
all_text.loc[ind_main_entries,"Year"]= year_vals
all_text.head(10)

,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head,Year
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1,NaN
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1,"1853, Septe-1857"
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1,NaN
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1,1862-1864
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,NaN,0,NaN
5,Abbott & Company,"Abbott & Company Papers, 1856-1871 Philadelphia, Pennsylvania a Section A 66 items OCT 10 ""49",4,5,Abbott & Company,1,1856-1871
6,Abbott & Company,"Abderhalden, Emil Papers, 1919 Halle, Germany Josiah C. Trent Collection in the History of Medicine--, Div, 4-8-60",5,6,Abbott & Company,1,1919
7,Abbott & Company,"Abbott & Company. Papers 1856-1871 Philadelphia, Pennsylvania, € Miscellaneous letters concerning scales sold by Abbott & Company.",6,7,Abbott & Company,1,1856-1871
8,"Abel, Ernest L.","Abel, Ernest L. Papers, 1922-1952 West Palm Beach, Palm Beach Co., Fla. Saat; 550 items & 8 vols. Box 1 of Labor Archives",7,8,"Abel, Ernest L.",1,1922-1952
9,"Abel, Ernest L.","Abel, Ernest L. Papers. West Palm Beach, Palm Beach Co., Fla. _’ : Ernest L. Abel was involved in postal union activity at least from the mid-1920s to the early 1950s. He served from 1925-1926 as Secretary-Treasurer for Local Union No. 749 of the National Federation of Post Office Clerks (AFL) and was State Representative for the National Federation at the same time. In 1927, Abel became the first president of the Florida Federation of Post Office Clerks, His union service included being President Union No. 749 in 1945, and State Legislative Representative of the’ ® Florida",7,9,NaN,0,NaN


In [28]:
# Start and End date column
start,end=[],[]
for i in ind_main_entries:
    year = all_text.Year[i]
    if type(year) == float and pd.isna(year):
        start.append(np.nan)
        end.append(np.nan)
        continue
    if(re.match(first,year)):
        start.append(int(year.split("-")[0]))
        if(len(year.split("-")[1])==2):
            end.append(int(year.split("-")[0][:2]+ year.split("-")[1]))   
        else:
            end.append(int(year.split("-")[1]))     
    elif(re.match(second,year)):
        # print("Second",year)
        lt =re.findall("1\d{3}",year)
        if(len(lt)==2):
            start.append(int(lt[0]))
            end.append(int(lt[1]))
        else:
            start.append(np.nan)
            end.append(np.nan)
    elif(re.match(third,year)):
        # print("Third",year)
        start.append(int(year))
        end.append(np.nan)
    else:
        start.append(np.nan)
        end.append(np.nan)        
all_text.loc[ind_main_entries, "Start"]  = start
all_text.loc[ind_main_entries, "End"]  = end
all_text.head()

,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head,Year,Start,End
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1,NaN,NaN,NaN
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1,"1853, Septe-1857",1853.0,1857.0
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1,NaN,NaN,NaN
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1,1862-1864,1862.0,1864.0
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,NaN,0,NaN,NaN,NaN


In [29]:
# Get locations
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
loc_vals = []

for i in main_entries.index:
    doc = nlp(str(main_entries.Text[i]))
    l = list()
    st= set()
    for ent in doc.ents:
        if(ent.text in st):
            continue
        if (ent.label_ == 'GPE'):
            l.append(ent.text)
            st.add(ent.text)
        elif (ent.label_ == 'LOC'):
            l.append(ent.text)
            st.add(ent.text)
    loc_vals.append(", ".join(l))
all_text["Loc"] =np.nan
all_text.loc[ind_main_entries,"Loc"]= loc_vals
all_text.head()

,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head,Year,Start,End,Loc
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1,NaN,NaN,NaN,Pa.
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1,"1853, Septe-1857",1853.0,1857.0,"Nashville, Tenne"
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1,NaN,NaN,NaN,"Abbeville District, South Carolina"
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1,1862-1864,1862.0,1864.0,"Abbott, Va."
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,NaN,0,NaN,NaN,NaN,NaN


In [30]:
# Get linke identifiers, size of each drawer and drawer numbers 
identifier_list = ['rubensteinmanuscriptcatalog_A_to_Amer', 'rubensteinmanuscriptcatalog_Ames_to_Armh', 'rubensteinmanuscriptcatalog_Armi_to_Bal', 'rubensteinmanuscriptcatalog_Bam_to_Bedh', 'rubensteinmanuscriptcatalog_Bedi_to_Bem', 'rubensteinmanuscriptcatalog_Ben_to_Blac', 'rubensteinmanuscriptcatalog_Blad_to_Q._Boyd', 'rubensteinmanuscriptcatalog_R._Boyd_to_I._Brown', 'rubensteinmanuscriptcatalog_J._Brown_to_L._Butler', 'rubensteinmanuscriptcatalog_M._Butler_to_Caq', 'rubensteinmanuscriptcatalog_Car_to_Chaq', 'rubensteinmanuscriptcatalog_Char_to_Clav', 'rubensteinmanuscriptcatalog_Claw_to_Com', 'rubensteinmanuscriptcatalog_Con_to_Conk', 'rubensteinmanuscriptcatalog_Conl_to_Crd', 'rubensteinmanuscriptcatalog_Cre_to_I._Davis', 'rubensteinmanuscriptcatalog_J._Davis_to_Dek', 'rubensteinmanuscriptcatalog_Del_to_Dov', 'rubensteinmanuscriptcatalog_Dow_to_Durg', 'rubensteinmanuscriptcatalog_Durh_to_Els', 'rubensteinmanuscriptcatalog_Elt_to_Fie', 'rubensteinmanuscriptcatalog_Fif_to_Frn', 'rubensteinmanuscriptcatalog_Fro_to_Geq', 'rubensteinmanuscriptcatalog_Ger_to_Gorl', 'rubensteinmanuscriptcatalog_Gorm_to_Grd', 'rubensteinmanuscriptcatalog_Gre_to_Grh', 'rubensteinmanuscriptcatalog_Gri_to_Hand', 'rubensteinmanuscriptcatalog_Hane_to_Harv', 'rubensteinmanuscriptcatalog_Harw_to_Hem', 'rubensteinmanuscriptcatalog_Hen_to_Holc', 'rubensteinmanuscriptcatalog_Hold_to_Huba', 'rubensteinmanuscriptcatalog_Hubb_to_I._Jackson', 'rubensteinmanuscriptcatalog_J._Jackson_to_I._Johnson', 'rubensteinmanuscriptcatalog_J._Johnson_to_Jz', 'rubensteinmanuscriptcatalog_K_to_Kira', 'rubensteinmanuscriptcatalog_Kirb_to_Lano', 'rubensteinmanuscriptcatalog_Lanp_to_Ler', 'rubensteinmanuscriptcatalog_Les_to_Lowq', 'rubensteinmanuscriptcatalog_Lowr_to_Mack', 'rubensteinmanuscriptcatalog_Macl_to_Manh', 'rubensteinmanuscriptcatalog_Mani_to_Maw', 'rubensteinmanuscriptcatalog_Max_to_Metg', 'rubensteinmanuscriptcatalog_Meth_to_Mh', 'rubensteinmanuscriptcatalog_Mi_to_Mord', 'rubensteinmanuscriptcatalog_More_to_Mur', 'rubensteinmanuscriptcatalog_Mus_to_Nn', 'rubensteinmanuscriptcatalog_No_to_Oz', 'rubensteinmanuscriptcatalog_P_to_Peo', 'rubensteinmanuscriptcatalog_Pep_to_Pn', 'rubensteinmanuscriptcatalog_Po_to_Puk', 'rubensteinmanuscriptcatalog_Pul_to_Rh', 'rubensteinmanuscriptcatalog_Ri_to_Rooj', 'rubensteinmanuscriptcatalog_Rook_to_Sam', 'rubensteinmanuscriptcatalog_San_to_Sem', 'rubensteinmanuscriptcatalog_Sen_to_Simo', 'rubensteinmanuscriptcatalog_Simp_to_Wh._Smith', 'rubensteinmanuscriptcatalog_Wi._Smith_to_So', 'rubensteinmanuscriptcatalog_Sp_to_Std', 'rubensteinmanuscriptcatalog_Ste_to_Sv', 'rubensteinmanuscriptcatalog_Sw_to_I._Thomas', 'rubensteinmanuscriptcatalog_J._Thomas_to_Tom', 'rubensteinmanuscriptcatalog_Ton_to_Tz', 'rubensteinmanuscriptcatalog_U', 'rubensteinmanuscriptcatalog_V_to_Ward', 'rubensteinmanuscriptcatalog_Ware_to_H._White', 'rubensteinmanuscriptcatalog_I._White_to_R._Williams', 'rubensteinmanuscriptcatalog_S._Williams_to_Wood', 'rubensteinmanuscriptcatalog_T._Wood_to_Wz', 'rubensteinmanuscriptcatalog_X_to_Z']
real_size = [838, 680, 935, 720, 639, 787, 765, 776, 730, 681, 707, 777, 798, 645, 790, 741, 654, 760, 581, 658, 772, 766, 757, 782, 660, 704, 761, 780, 736, 737, 750, 923, 730, 706, 668, 691, 720, 753, 844, 686, 713, 339, 604, 787, 751, 764, 796, 858, 762, 877, 839, 833, 778, 823, 653, 662, 692, 702, 873, 733, 713, 689, 659, 774, 815, 734, 727, 543, 284]
drawer_no = ['157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '205', '206', '207', '208',
 '209', '210', '211', '212', '213', '214', '215', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '229', '230', '231']

In [31]:
# create link list for the dataset
link_identifier_list = ["https://archive.org/details/" + i  for i in identifier_list]
link_list = []
for i in range(len(link_identifier_list)):
    curr_size = real_size[i]
    curr_ident = link_identifier_list[i]
    # print(curr_size,curr_ident)
    for i in range(curr_size):
        link_list.append(curr_ident+"/page/n{}".format(i)) 


drawer_no_list = []
for i in range(len(real_size)):
    for j in range(real_size[i]):
        drawer_no_list.append(drawer_no[i])   
all_text["Link"] = link_list
all_text["Drawer_No"] = drawer_no_list

In [32]:
all_text.head()

,Name,Text,Collection,Page_No_Within_Drawer,Collection_Head,Is_collection_head,Year,Start,End,Loc,Link,Drawer_No
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",0,0,A. B. Davis and Company,1,NaN,NaN,NaN,Pa.,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n0,157
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",1,1,A.H. Roscoe,1,"1853, Septe-1857",1853.0,1857.0,"Nashville, Tenne",https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n1,157
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,2,2,Abbeville District,1,NaN,NaN,NaN,"Abbeville District, South Carolina",https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n2,157
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",3,3,"Abbott, William B",1,1862-1864,1862.0,1864.0,"Abbott, Va.",https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n3,157
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",3,4,NaN,0,NaN,NaN,NaN,NaN,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n4,157


In [35]:
all_text.set_index("Collection_Head",inplace=True)
all_text.rename(columns = {"Page_No_Within_Drawer":"Page_drawer","Is_collection_head":"Coll_head"},inplace=True)
all_text = all_text[["Name","Text","Year","Loc","Start","End", "Page_drawer","Drawer_No","Link",'Coll_head',"Collection"]]

In [36]:
all_text.head(2)

,Name,Text,Year,Loc,Start,End,Page_drawer,Drawer_No,Link,Coll_head,Collection
Collection_Head,,,,,,,,,,,
A. B. Davis and Company,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",NaN,Pa.,NaN,NaN,0,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n0,1,0
A.H. Roscoe,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme","1853, Septe-1857","Nashville, Tenne",1853.0,1857.0,1,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n1,1,1


In [37]:
all_text.to_csv("all_sorted_collection.csv")

In [4]:
df = pd.read_csv("all_sorted_collection.csv")
df = df[~df.Text.isnull()]
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stop_words.extend(["papers","letters",
"items","letter","collection","family","co","also","added","see","sketch","one","two","ten", "pp","book","section","ndhyme","many","item","next"])
stop_words.extend(['dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov',
"december","january","febraury","march","april","may","june","july","august","september","october","november","vol","volume","addition","sept","guide"])
def clean(row):
    if(type(row))==float:
        return row
# remove punctuation
    full_no_punc =re.sub('[%s]' % re.escape(string.punctuation+string.digits), '', row)
    full_no_punc = " ".join([i for i in full_no_punc.split() if len(i)>1])
    
    # Tokenize
    text_tokens = word_tokenize(full_no_punc)
    # # Change to lower case and select only alphabet
    pre_process = [i.lower() for i in text_tokens if i.isalpha() and i.lower() not in stop_words]
    return " ".join(pre_process)
df["Clean"] = df.Text.apply(clean)
df = df.iloc[:,[1,2,-1]]
df.head()

,Name,Text,Clean
0,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company",davis company philadelphia pa davis company
1,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme",aehe roscoe firm nashville tenne journal septe dece ppe wholesale retail druggist dealer paints oils dyestuffse summary journal account documents sale chemical herbal drugs paint painting supplies dyestuffs personal household supplies garden seeds individuals businessese le drugstorestennesseee paint shopsequipment tennesseee dyes dyeinge household suppliestennesseee herbstherapeutic usee seed industry tradetennessees nashville nne qo
2,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,abbeville district south carolina abbeville district
3,"Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items",abbott william frederick va
4,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.",abbott william fre gerick va william abbott evidently welltodo farmer frederick va several documents concerned evaluation damage done property troops andreceipts hay bought abbott various times army


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer

#get the text column 
docs=df.Clean.tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,max_features=10000)
word_count_vector=cv.fit_transform(docs)


tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)


TfidfTransformer()

In [6]:
docs_test=df['Clean'].tolist()
feature_names=cv.get_feature_names()

In [9]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

def get_keywords(idx, docs):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    return list(keywords.keys())[:5]

def get_top_words(row):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([row]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    return list(keywords.keys())
li = df.Clean.apply(get_top_words)

In [22]:
d = pd.read_csv("all_sorted_collection.csv")
d["Key_word"] = np.nan
ind = d[~d.Text.isnull()].index
d.loc[ind,"Key_word"] = li
d = d.iloc[:,[0,1,2,-1,3,4,5,6,7,8,9,10]]
d.set_index("Collection_Head",inplace=True)

In [23]:
d.head(20)

,Name,Text,Key_word,Year,Loc,Start,End,Page_drawer,Drawer_No,Link,Coll_head
Collection_Head,,,,,,,,,,,
A. B. Davis and Company,A. B. Davis and Company,"A. B. Davis and Company (Philadelphia, Pa. ) See Davis (A. B.) and Company","[davis, company, philadelphia, pa]",NaN,Pa.,NaN,NaN,0,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n0,1
A.H. Roscoe,A.H. Roscoe,"AeHe Roscoe (Firm: Nashville, Tenne )e Journal, 1853, Septe-1857, Dece 1 item( 800 ppe )e Wholesale and retail druggist and dealer in paints, oils, and dyestuffse Summary: Journal (account book ) documents the sale of chemical and herbal drugs, paint and painting supplies, dyestuffs, personal and household supplies, and garden seeds to individuals and businessese le Drugstores--Tennesseee 2e Paint shops--Equipment and supplies-—- Tennesseee 3e Dyes and dyeinge 46 Household supplies--Tennesseee 5c Herbs--Therapeutic usee 6+ Seed industry and trade--Tennessees 7e Nashville ite® nne )—--Commercee 20 MAY QO1 23804905 NDHYme","[paint, household, nashville, supplies, journal, dyes, roscoe, seed, qo, seeds]","1853, Septe-1857","Nashville, Tenne",1853.0,1857.0,1,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n1,1
Abbeville District,Abbeville District,Abbeville District (8.C.) See South Carolina. Abbeville District,"[abbeville, district, south, carolina]",NaN,"Abbeville District, South Carolina",NaN,NaN,2,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n2,1
"Abbott, William B","Abbott, William B","Abbott, William B. Papers, 1862-1864 Frederick Co., Va. Section A 5-16-57 GUIDE 10 items","[abbott, frederick, va, william]",1862-1864,"Abbott, Va.",1862.0,1864.0,3,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n3,1
NaN,"Abbott, William B","Abbott, William B. Papers, 1862-1864, Fre- Gerick Co., Va. 10 items. Sketch These are the papers of William B. Abbott, evidently a well-to-do farmer of Frederick Co., Va. There are several documents concerned with the evaluation of damage done to his property by C. S. A. troops in 1862, and.receipts in 1864 for hay bought from Abbott at various times in Aug., 1864 by the C. S. A. Army.","[abbott, evaluation, damage, fre, hay, va, bought, done, farmer, william]",NaN,NaN,NaN,NaN,4,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n4,0
Abbott & Company,Abbott & Company,"Abbott & Company Papers, 1856-1871 Philadelphia, Pennsylvania a Section A 66 items OCT 10 ""49","[abbott, pennsylvania, philadelphia, company]",1856-1871,"Philadelphia, Pennsylvania",1856.0,1871.0,5,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n5,1
Abbott & Company,Abbott & Company,"Abderhalden, Emil Papers, 1919 Halle, Germany Josiah C. Trent Collection in the History of Medicine--, Div, 4-8-60","[emil, germany, div, medicine, trent, josiah, history]",1919,"Abderhalden, Halle, Germany",1919.0,NaN,6,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n6,1
Abbott & Company,Abbott & Company,"Abbott & Company. Papers 1856-1871 Philadelphia, Pennsylvania, € Miscellaneous letters concerning scales sold by Abbott & Company.","[abbott, company, scales, sold, pennsylvania, philadelphia, miscellaneous, concerning]",1856-1871,"Philadelphia, Pennsylvania",1856.0,1871.0,7,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n7,1
"Abel, Ernest L.","Abel, Ernest L.","Abel, Ernest L. Papers, 1922-1952 West Palm Beach, Palm Beach Co., Fla. Saat; 550 items & 8 vols. Box 1 of Labor Archives","[palm, beach, abel, ernest, fla, box, archives, labor, west, vols]",1922-1952,"West Palm Beach, Palm Beach Co., Fla.",1922.0,1952.0,8,157,https://archive.org/details/rubensteinmanuscriptcatalog_A_to_Amer/page/n8,1


In [24]:
d.to_csv("all_sorted_keyword.csv")